<a href="https://colab.research.google.com/github/micattia/Capstone/blob/master/05_Exprimental_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#load pandas component for data science
import pandas as pd

#load Apple data 
linkAP1 = 'https://raw.githubusercontent.com/micattia/Capstone/master/AppleFinalData.csv'
dfAPPL = pd.read_csv(linkAP1)
linkAP2 = 'https://raw.githubusercontent.com/micattia/Capstone/master/AppleNewsStock.csv'
dfAPPLnews = pd.read_csv(linkAP2)

#load Microsoft data
linkMS1 = 'https://raw.githubusercontent.com/micattia/Capstone/master/MicrosoftFinalData.csv'
dfMS = pd.read_csv(linkMS1)
linkMS2 = 'https://raw.githubusercontent.com/micattia/Capstone/master/MicrosoftNewsStock.csv'
dfMSnews = pd.read_csv(linkMS2)

# Changing Some column names for the Microsoft main data frame to:
# 1- removed the space from the Adj Close column, so that it would not give errors with coding
# 2- standarised all columns to start with caps lock.

dfMS = dfMS.rename(index=str, columns={"Adj Close": "Adj_Close", "compound": "Compound", "neg": "Neg", "neu": "Neu", "pos": "Pos"})

# Changing Some column names for the Apple main data frame to:
# 1- removed the space from the Adj Close column, so that it would not give errors with coding
# 2- standarised all columns to start with caps lock.

dfAPPL = dfAPPL.rename(index=str, columns={"Adj Close": "Adj_Close", "compound": "Compound", "neg": "Neg", "neu": "Neu", "pos": "Pos"})


In [0]:
#loading required libraries 
#==========================
from statsmodels.graphics.gofplots import qqplot
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns
import plotly.plotly as py
import plotly.figure_factory as ff
%matplotlib inline

pyplot.style.use('ggplot')

In [4]:
from statsmodels.formula.api import ols
model = ols('Adj_Close ~ Open + High + Low + Close + Compound', data=dfMS)
model = model.fit()
print(model.params)
#print(model)

Intercept   -4.535261e-14
Open        -6.522560e-16
High         1.665335e-16
Low         -2.470246e-15
Close        1.000000e+00
Compound    -1.453698e-15
dtype: float64


In [5]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj_Close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.677e+29
Date:                Tue, 02 Jul 2019   Prob (F-statistic):               0.00
Time:                        02:39:42   Log-Likelihood:                 69641.
No. Observations:                2517   AIC:                        -1.393e+05
Df Residuals:                    2511   BIC:                        -1.392e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.535e-14   1.67e-14     -2.718      0.0

In [6]:
# Using Anova test to 
import statsmodels.api as sm
#from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model1 = ols('Adj_Close ~ Open + High + Low + Close + Compound', data=dfMS).fit()
model2 = ols('Adj_Close ~ Open + High + Low + Close + Compound + Neg + Neu + Pos', data=dfMS).fit()
anova_result = anova_lm(model2)
print(anova_result)

              df        sum_sq       mean_sq             F    PR(>F)
Open         1.0  2.632082e+05  2.632082e+05  1.108156e+31  0.000000
High         1.0  2.442461e+02  2.442461e+02  1.028322e+28  0.000000
Low          1.0  1.248891e+02  1.248891e+02  5.258068e+27  0.000000
Close        1.0  1.063385e+02  1.063385e+02  4.477053e+27  0.000000
Compound     1.0  2.523816e-27  2.523816e-27  1.062574e-01  0.744473
Neg          1.0  2.180338e-28  2.180338e-28  9.179631e-03  0.923679
Neu          1.0  2.268480e-27  2.268480e-27  9.550729e-02  0.757315
Pos          1.0  3.304157e-27  3.304157e-27  1.391112e-01  0.709198
Residual  2508.0  5.956979e-23  2.375191e-26           NaN       NaN


In [7]:
#Testing the Simple Linier Regression model and see its predective values
# in this section we created the model and did predection using some data input by hand 
model = ols('Adj_Close ~ Open + High + Low + Close + Compound + Neg + Neu + Pos', data=dfMS).fit()
test_df = pd.DataFrame(columns=["Open" ,"High","Low","Close","Compound","Neg","Neu","Pos"], data=[[30.14, 30.23, 30.03, 30.19, 0.126, 0.048,0.868,0.084]])
mpg_pred = model.get_prediction(test_df)
mpg_pred.conf_int(alpha=0.05)

array([[30.19, 30.19]])

In [11]:
################################################################################
# BUILDING FIRST PREDECTION MODEL - SIMPLE LINIER REGRESSION
# SPLITING THE DATA INTO TRAINING SET AND TEST SET
# THEN BUILDONG THE MODEL BASED ON THE TRAINING SET
################################################################################
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import statsmodels.discrete.discrete_model as sm
import statsmodels.api as sm1

train_set, test_set = train_test_split(dfMS, train_size=0.7, random_state=1)

y_train, X_train = dmatrices('Adj_Close ~ Open + High + Low + Close + Compound + Neg + Neu + Pos', train_set, return_type = 'dataframe')
linier = sm1.OLS(y_train , X_train)
linreg = linier.fit()
linreg.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj_Close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.113e+31
Date:                Tue, 02 Jul 2019   Prob (F-statistic):               0.00
Time:                        03:13:55   Log-Likelihood:                 52172.
No. Observations:                1761   AIC:                        -1.043e+05
Df Residuals:                    1752   BIC:                        -1.043e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.057e-14   2.95e-15     -3.580      0.000   -1.64e-14   -4.78e-15
Open       -3.275e-15   3.69e-15     -0.887      0.375   -1.05e-14    3.96e-15
High        1.277e-15   3.99e-15      0.320      0.749   -6.55e-15    9.11e-15
Low         4.226e-15   3.94e-15      1.073      0.283    -3.5e-15    1.19e-14
Close          1.0000    3.8e-15   2.63e+14      0.000       1.000       1.000
Compound    4.552e-15   3.87e-15      1.175      0.240   -3.04e-15    1.21e-14
Neg         3.209e-14   2.68e-14      1.196      0.232   -2.05e-14    8.47e-14
Neu         -1.18e-15   2.42e-15     -0.488      0.626   -5.92e-15    3.56e-15
Pos        -1.788e-14   2.33e-14     -0.768      0.443   -6.36e-14    2.78e-14
==============================================================================
Omnibus:                      172.856   Durbin-Watson:                   0.305
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              226.887
Skew:                          -0.879   Prob(JB):                     5.40e-50
Kurtosis:                       2.997   Cond. No.                     2.86e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.86e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
# PREDECTING THE TEST SET BASED ON THE SIMPLE LINEAR REGRESSION MODEL CREATED BY THE TRAINING SET 
y_test, X_test = dmatrices('Adj_Close ~ Open + High + Low + Close + Compound + Neg + Neu + Pos', test_set, return_type = 'dataframe')
pred = linreg.predict(X_test)
#pred = model.get_prediction(test_df)

In [13]:
pred

2088    42.86
1511    26.37
228     35.57
1974    45.53
2297    50.79
1603    28.79
1299    29.53
2205    43.89
1881    40.19
195     29.16
107     30.78
992     26.94
2191    47.27
1481    29.50
700     25.20
544     17.83
2166    45.62
1308    30.58
403     25.25
2266    54.20
569     16.48
415     26.11
2086    42.29
2058    42.41
1333    31.99
2314    50.50
1489    29.52
790     29.32
2279    55.35
870     28.24
        ...  
2310    49.41
1139    24.04
1032    28.60
1578    27.92
1856    39.94
1629    34.27
1323    31.56
418     25.44
2320    51.18
110     30.97
223     30.90
1705    32.69
1664    36.27
229     36.81
1742    35.94
845     30.82
627     21.40
1370    31.16
2302    51.22
1232    27.19
1441    30.54
767     30.36
583     17.48
1886    40.29
351     28.99
1477    29.20
1958    46.24
1349    30.98
2223    45.57
1490    29.50
Length: 756, dtype: float64